# Libraries

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib
from pathlib import Path
import json

import csv
import pandas as pd

from utils.utils import get_slice, extract_traces
from annotation.annotations_io import load_annotations, save_annotations

# Functions

In [ ]:
def load_flow_file(flow_file_path):
    flow, index = [], []

    if os.path.exists(flow_file_path):
        with open(flow_file_path, "r") as f:
            data = csv.reader(f)
            for row in data:
                flow.append(row[1])
                index.append(int(row[0]))
    else:
        flow = ["none"]
        index = [0]

    return flow, index

def get_ids(dataset_path):
    A, W = load_annotations(Path(dataset_path)/"processed")
    return list(W.df["id"]), list([name.decode() for name in W.df["name"]])

def get_flow(dataset):
    flow_file = os.path.join(dataset, "processed", "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    return frames, flow

def load_traces(dataset, **kwargs):
    
    
    traces_folder = os.path.join(dataset,"processed", "traces")
    
    if 'trace_file' in kwargs.keys():
        traces_file = os.path.join(traces_folder, kwargs['trace_file'])
        
    elif os.path.isfile(os.path.join(traces_folder, "traces.npy")):
        traces_file = os.path.join(traces_folder, "traces.npy")
        
    elif os.path.isfile(os.path.join(traces_folder, "tracked_traces.npy")):
        traces_file = os.path.join(traces_folder, "tracked_traces.npy")
        
    else:
        traces_file = os.path.join(traces_folder, "not_tracked_traces.npy")
        
    return np.load(traces_file)

def get_odor_trace(dataset, odor, **kwargs):
    (frames,flow) = get_flow(dataset)
    traces = load_traces(dataset, **kwargs)
    odor_idx = (int(frames[flow.index(odor)]), int(frames[flow.index(odor)+1]))
    
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

def get_neuron_traces(paths, neuron, odor, **kwargs):

    shape = get_odor_trace(paths[0], odor, **kwargs)[0].shape
    neuron_traces = []
    
    for path in paths:
        (ids, names) = get_ids(path)
        try:
            idx = ids[names.index(neuron)]
            path_trace = get_odor_trace(path, odor, **kwargs)[idx]
            neuron_traces.append(path_trace)
            print(path, path_trace.shape)
        except ValueError:
            empty_trace = np.zeros(shape)
            empty_trace[:] = np.nan
            neuron_traces.append(empty_trace)
            
    neuron_traces = np.stack(neuron_traces, 0)
    return neuron_traces

def get_all_neuron_traces(paths, all_names, odor, **kwargs):
    odor_traces = []
    for name in all_names:
        odor_traces.append(get_neuron_traces(paths, name, odor), **kwargs)
    return odor_traces


buffer_idx = [60, 60] 
colors_dict = {
    "Control": "#B2BABB",
    "Pheromone": "#F7DC6F"
}

# AIB

In [ ]:
aib_odors = ['Pheromone', 'Control']
aib_names = ['AIBL', 'AIBR']
aib_paths = [
    Path(r"X:\Maede\data\daure\20211008\5"),
    Path(r"X:\Maede\data\daure\20211008\5_2"),
    Path(r"X:\Maede\data\daure\20211008\6"),
    Path(r"X:\Maede\data\daure\20211008\7"),
    Path(r"X:\Maede\data\daure\20211008\8"),
    Path(r"X:\Maede\data\daure\20211008\8_2"),
    Path(r"X:\Maede\data\daure\20211008\9"),
    Path(r"X:\Maede\data\daure\20211008\9_2"),
    Path(r"X:\Maede\data\daure\20211008\10"),
    Path(r"X:\Maede\data\daure\20211008\10_2"),
    Path(r"X:\Maede\data\daure\20211008\11_2"),
    Path(r"X:\Maede\data\daure\20211008\12"),
    Path(r"X:\Maede\data\daure\20211008\12_2")]

for path in aib_paths:
    extract_traces(path)

def get_odor_trace_pher(dataset, odor):
    p = Path(dataset)/'traces/traces.npy'
    traces = np.load(p)
    flow_file = os.path.join(dataset, "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    odor_idx = (int(frames[flow.index(odor)]), int(frames[flow.index(odor)+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

def get_trace_pher(dataset):
    p = Path(dataset)/'traces/traces.npy'
    traces = np.load(p)
    flow_file = os.path.join(dataset, "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    odor_idx = (int(frames[flow.index('Pheromone')]), int(frames[flow.index('Control')+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

def get_trace_process(dataset):
    p = Path(dataset)/'traces/processes_traces.npy'
    traces = np.load(p)
    flow_file = os.path.join(dataset, "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    odor_idx = (int(frames[flow.index('Pheromone')]), int(frames[flow.index('Control')+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[0, t_idx]

times = list(range(buffer_idx[0]+buffer_idx[1]+240))
df = pd.DataFrame()
dic = {}

for path in aib_paths:
    path = Path(path)/'processed'
    A, W = load_annotations(path)
    names = [name.decode() for name in W.df["name"]]
    flow_file = os.path.join(path, "flow_file.txt")
    flow, _ = load_flow_file(flow_file)
    flow = [f  for f in flow if f!='CTX']
    dic['animal'] = str(path).split('\\')[-2]
    
    
    trace = get_trace_process(path)

    dic['neuron'] = 'AIB_process'
    
    bl = np.nanmean(trace[buffer_idx[0]-20:buffer_idx[0]])
    trace = (trace-bl)/bl * 100
    for t in times:
        dic[t] = [trace[t]]

    df = df.append(pd.DataFrame(dic), ignore_index=True)
    
df.to_csv(Path(r"X:\Maede\data\daure\20211008\traces_process.csv"))

times = list(range(buffer_idx[0]+buffer_idx[1]+240))
df = pd.DataFrame()
dic = {}

for path in aib_paths:
    path = Path(path)/'processed'
    A, W = load_annotations(path)
    names = [name.decode() for name in W.df["name"]]
    flow_file = os.path.join(path, "flow_file.txt")
    flow, _ = load_flow_file(flow_file)
    flow = [f  for f in flow if f!='CTX']
    dic['animal'] = str(path).split('\\')[-2]
    
    
    traces = get_trace_pher(path)

    for neuron in aib_names:
        dic['neuron'] = neuron
        try:
            trace = traces[names.index(neuron)]
        except ValueError:
            trace = np.zeros(traces.shape[-1])
            trace[:] = np.nan
        bl = np.nanmean(trace[buffer_idx[0]-20:buffer_idx[0]])
        trace = (trace-bl)/bl * 100
        for t in times:
            dic[t] = [trace[t]]

        df = df.append(pd.DataFrame(dic), ignore_index=True)

df.to_csv(Path(r"X:\Maede\data\daure\20211008\traces_soma.csv"))

# AIA

In [ ]:
aia_odors = ['Pheromone', 'Control']
aia_main_path = Path(r"X:\Mahdi\Cynthia Chai\PS9111\microfluidics")
aia_paths = []

for p1 in os.listdir(aia_main_path):
    if os.path.isdir(os.path.join(aia_main_path, p1)):
        for p2 in os.listdir(os.path.join(aia_main_path, p1)):
            for p3 in os.listdir(os.path.join(aia_main_path, p1, p2)):
                aia_paths.append(os.path.join(aia_main_path, p1, p2, p3))

for path in aib_paths:
    extract_traces(path)
    

def get_trace_aia(dataset):
    p = Path(dataset)/'processed/traces/processes_traces.npy'
    traces = np.load(p)
    (frames,flow) = get_flow(dataset)
    odor_idx = (int(frames[flow.index('Pheromone')]), int(frames[flow.index('Control')+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[0, t_idx]

def get_trace_aia_soma(dataset):
    p = Path(dataset)/'processed/traces/traces.npy'
    traces = np.load(p)
    (frames,flow) = get_flow(dataset)
    odor_idx = (int(frames[flow.index('Pheromone')]), int(frames[flow.index('Control')+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

times = list(range(buffer_idx[0]+buffer_idx[1]+240))
df = pd.DataFrame()
dic = {}
for path in aia_paths[:]:
    _, flow = get_flow(path)
    flow = [f  for f in flow if f!='CTX']
    animal_path, run = os.path.split(path)
    _, dic['animal'] = os.path.split(animal_path)
    dic['animal'] = dic['animal'][7:]
    dic['run'] = run[4:]
    dic['neuron'] = 'AIA process'
        
   
    trace = get_trace_aia(path)
    bl = np.nanmean(trace[buffer_idx[0]-20:buffer_idx[0]])
    trace = (trace-bl)/bl * 100
    for t in times:
        dic[t] = [trace[t]]

    df = df.append(pd.DataFrame(dic), ignore_index=True)
    
df.to_csv(Path(r"X:\Mahdi\Cynthia Chai\PS9111\microfluidics\process_traces.csv"))

times = list(range(buffer_idx[0]+buffer_idx[1]+240))
df = pd.DataFrame()
dic = {}

for path in aia_paths:
    print(path)
    A, W = load_annotations(Path(path)/'processed')
    names = [name.decode() for name in W.df["name"]]
    _, flow = get_flow(path)
    flow = [f  for f in flow if f!='CTX']
    animal_path, run = os.path.split(path)
    _, dic['animal'] = os.path.split(animal_path)
    dic['animal'] = dic['animal'][7:]
    dic['run'] = run[4:]
    
    traces = get_trace_aia_soma(Path(path))

    for neuron in ['AIAL', 'AIAR']:
        dic['neuron'] = neuron
        try:
            trace = traces[names.index(neuron)]
        except ValueError:
            trace = np.zeros(traces.shape[-1])
            trace[:] = np.nan
        bl = np.nanmean(trace[buffer_idx[0]-20:buffer_idx[0]])
        trace = (trace-bl)/bl * 100
        for t in times:
            dic[t] = [trace[t]]

        df = df.append(pd.DataFrame(dic), ignore_index=True)
        
df.to_csv(Path(r"X:\Mahdi\Cynthia Chai\PS9111\microfluidics\soma_traces.csv"))

# ASK, ADL, AIA

In [ ]:
pher_odors = ['Pheromone', 'Control']
neuron_names = ['ASKL', 'ADLL', 'AIAL', 'ASKR', 'ADLR', 'AIAR', 'AIA']

main_path = Path(r"Y:\Mahdi\Cynthia Chai\PS8955\microfluidics")
pher_paths = [os.path.join(main_path, r"12_15_2020\animal_6\run_1\tracked"),
             os.path.join(main_path, r"12_15_2020\animal_5\run_1\tracked"),
             os.path.join(main_path, r"12_15_2020\animal_4\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_2\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_5\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_8\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_10\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_11\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_12\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_13\run_1\tracked"),
             os.path.join(main_path, r"12_22_2020\animal_14\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_3\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_8\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_11\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_12\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_13\run_1\tracked"),
             os.path.join(main_path, r"12_23_2020\animal_14\run_1\tracked")]

for path in aib_paths:
    extract_traces(path)

def get_odor_trace_pher(dataset, odor):
    p = Path(dataset)/'traces.npy'
    traces = np.load(p)[1]
    flow_file = os.path.join(dataset, "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    odor_idx = (int(frames[flow.index(odor)]), int(frames[flow.index(odor)+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

def get_trace_pher(dataset):
    p = Path(dataset)/'traces.npy'
    traces = np.load(p)[1]
    flow_file = os.path.join(dataset, "flow_file.txt")
    flow, frames = load_flow_file(flow_file)
    odor_idx = (int(frames[flow.index('Pheromone')]), int(frames[flow.index('Control')+1]))
    t_idx = np.s_[odor_idx[0]-buffer_idx[0]:odor_idx[1]+buffer_idx[1]]
    return traces[:, t_idx]

times = list(range(buffer_idx[0]+buffer_idx[1]+240))
df = pd.DataFrame()
dic = {}

for path in pher_paths:
    A, W = load_annotations(Path(path))
    names = [name.decode() for name in W.df["name"]]
    flow_file = os.path.join(path, "flow_file.txt")
    flow, _ = load_flow_file(flow_file)
    flow = [f  for f in flow if f!='CTX']
    animal_path, run = os.path.split(path[:-8])
    date_path, dic['animal'] = os.path.split(animal_path)
    _, dic['date'] = os.path.split(date_path)
    dic['animal'] = dic['animal'][7:]
    dic['run'] = run[4:]
    
    traces = get_trace_pher(path)

    for neuron in neuron_names:
        dic['neuron'] = neuron
        try:
            trace = traces[names.index(neuron)]
        except ValueError:
            trace = np.zeros(traces.shape[-1])
            trace[:] = np.nan
        bl = np.nanmean(trace[buffer_idx[0]-20:buffer_idx[0]])
        trace = (trace-bl)/bl * 100
        for t in times:
            dic[t] = [trace[t]]

        df = df.append(pd.DataFrame(dic), ignore_index=True)
        
df.to_csv(Path(r"Y:\Mahdi\Cynthia Chai\PS8955\microfluidics\traces2.csv"))